<a href="https://colab.research.google.com/github/jeonggunlee/CUDATeaching/blob/master/01_cuda_lab/01_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple C Programming with Google Colab

- Instructir: Jeong-Gun Lee (School of Software, Hallym University)
- Email: jeonggun.lee@gmail.com
- Web: http://www.onchp.net


In [8]:
%%writefile prog.c

#include<stdio.h>

int main(void)
{
    printf("Hello World!~~~");
    
    return 0;  // 정상 완료
}

Overwriting prog.c


In [9]:
!ls

prog.c	sample_data


In [10]:
!gcc --version

gcc (Ubuntu 7.4.0-1ubuntu1~18.04) 7.4.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [0]:
!gcc -o hello prog.c

In [12]:
!./hello

Hello World!~~~